## Install and import libraries

In [4]:
!pip3 install pandas
!pip3 install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 321.9 kB/s eta 0:00:00 kB/s eta 0:00:01:01


In [38]:
import pandas as pd
import numpy as np

## 1. Extract the column names for each excel sheet

In [62]:
df_data_specs = pd.read_excel(file_path, sheet_name='Data Specs')
df_data_specs.drop(columns=["Unnamed: 1", "Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], inplace=True)

data_definitions = {}
for index, row in df_data_specs.drop(columns=["Report Layout"]).dropna().iterrows():
    data_definitions[row["Data Definitions:"]] = row["Unnamed: 6"]

page_columns_map = {}
page_map = []
page = ""
for index, row in df_data_specs.iterrows():
    if row["Report Layout"] is np.nan:
        if len(page_map) != 0:
            page_columns_map[page] = page_map
            page = ""
            page_map = []
    elif len(page_map) == 0 and page == "":
        page = row["Report Layout"].replace("Tab: ", "")
    else:
        page_map.append(row["Report Layout"])

## 2. Preprocessing for Tab: TIN_Q1-5_TIN_Info

### A. Load sheet

In [119]:
df_tin_info = pd.read_excel(file_path, sheet_name='TIN_Q1-5_TIN_Info')
df_tin_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1135 entries, 0 to 1134
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PERFORMANCE_YEAR  1135 non-null   int64  
 1   QRO               1135 non-null   object 
 2   TIN_CCN           1135 non-null   int64  
 3   ENROLLMENT_TYPE   1135 non-null   object 
 4   BENE_CNT          1135 non-null   object 
 5   AVR_RISK_SCORE    1113 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 53.3+ KB


### B. Check for duplicates

In [90]:
df_tin_info_before = df_tin_info.copy()
df_tin_info.drop_duplicates(inplace=True)
rows_before, rows_after = df_tin_info_before.shape[0], df_tin_info.shape[0]
if rows_before == rows_after:
    print("No duplicate rows found")
else:
    print(f"{rows_before - rows_after} duplicate rows found")

No duplicate rows found


### C. Convert ENROLLMENT TYPE column to onehot encoding

In [91]:
one_hot_encoded = pd.get_dummies(df_tin_info['ENROLLMENT_TYPE'], prefix='ENROLLMENT_TYPE')
df_tin_info_encoded = df_tin_info.join(one_hot_encoded)
df_tin_info_encoded.head()

,PERFORMANCE_YEAR,QRO,TIN_CCN,ENROLLMENT_TYPE,BENE_CNT,AVR_RISK_SCORE,ENROLLMENT_TYPE_Aged,ENROLLMENT_TYPE_Aged-Dual,ENROLLMENT_TYPE_Disabled,ENROLLMENT_TYPE_ESRD,ENROLLMENT_TYPE_Unknown
0,2022,MKLO,866020,ESRD,<11,1.861,False,False,False,True,False
1,2022,MKLO,866020,Unknown,109,0.736,False,False,False,False,True
2,2022,MKLO,866020,Aged-Dual,344,0.876,False,True,False,False,False
3,2022,MKLO,866020,Disabled,713,0.863,False,False,True,False,False
4,2022,MKLO,866020,Aged,2868,0.940,True,False,False,False,False


### D. Convert QRO column to onehot encoding

In [92]:
one_hot_encoded = pd.get_dummies(df_tin_info_encoded['QRO'], prefix='QRO')
df_tin_info_encoded = df_tin_info_encoded.join(one_hot_encoded)
df_tin_info_encoded.head()

,PERFORMANCE_YEAR,QRO,TIN_CCN,ENROLLMENT_TYPE,BENE_CNT,AVR_RISK_SCORE,ENROLLMENT_TYPE_Aged,ENROLLMENT_TYPE_Aged-Dual,ENROLLMENT_TYPE_Disabled,ENROLLMENT_TYPE_ESRD,ENROLLMENT_TYPE_Unknown,QRO_MKLO,QRO_MTIP,QRO_NTVP,QRO_OWTS,QRO_SDRU,QRO_UPLN
0,2022,MKLO,866020,ESRD,<11,1.861,False,False,False,True,False,True,False,False,False,False,False
1,2022,MKLO,866020,Unknown,109,0.736,False,False,False,False,True,True,False,False,False,False,False
2,2022,MKLO,866020,Aged-Dual,344,0.876,False,True,False,False,False,True,False,False,False,False,False
3,2022,MKLO,866020,Disabled,713,0.863,False,False,True,False,False,True,False,False,False,False,False
4,2022,MKLO,866020,Aged,2868,0.940,True,False,False,False,False,True,False,False,False,False,False


### E. Outlier detection for AVG RISK SCORE

In [93]:
Q1 = df_tin_info_encoded['AVR_RISK_SCORE'].quantile(0.25)
Q3 = df_tin_info_encoded['AVR_RISK_SCORE'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df_tin_info_encoded[(df_tin_info_encoded['AVR_RISK_SCORE'] < lower_bound) | (df_tin_info_encoded['AVR_RISK_SCORE'] > upper_bound)]

outliers[['PERFORMANCE_YEAR', 'QRO', 'TIN_CCN', 'ENROLLMENT_TYPE', 'AVR_RISK_SCORE']]

,PERFORMANCE_YEAR,QRO,TIN_CCN,ENROLLMENT_TYPE,AVR_RISK_SCORE
0,2022,MKLO,866020,ESRD,1.861
5,2023,MKLO,866020,Unknown,2.455
12,2019,MTIP,166870,Disabled,0.145
36,2020,MTIP,496926,Unknown,1.841
55,2021,MTIP,166870,Disabled,0.169
...,...,...,...,...,...
1016,2023,SDRU,636998,Disabled,3.635
1059,2023,SDRU,351739,Disabled,1.901
1126,2022,UPLN,430950,ESRD,1.689
1130,2023,UPLN,430950,Unknown,1.787


### F. Consistency checks

In [96]:
performance_year_check = df_tin_info_encoded['PERFORMANCE_YEAR'].between(2019, 2023).all()
tin_ccn_unique_check = df_tin_info_encoded['TIN_CCN'].is_unique

if performance_year_check:
    print("All performace years between 2019 and 2023")
else:
    print("Found inconsistency in performace years. Not all in range 2019 to 2023")

if tin_ccn_unique_check:
    print("All CCN values are unique")
else:
    print("Not all CCN values are unique")

All performace years between 2019 and 2023
Not all CCN values are unique


## 3. Preprocessing for Tab: TIN_Q7,8-Total_Cost_EnrollType

### A. Load sheet

In [118]:
df_total_cost_enroll = pd.read_excel(file_path, sheet_name='TIN_Q7,8-Total_Cost_EnrollType')
df_total_cost_enroll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PERFORMANCE_YEAR       375 non-null    int64  
 1   QRO                    375 non-null    object 
 2   TIN_CCN                375 non-null    int64  
 3   TOTAL_SPEND            375 non-null    float64
 4   AGED_TOTAL_SPEND       352 non-null    float64
 5   DISABLED_TOTAL_SPEND   327 non-null    float64
 6   ESRD_TOTAL_ESRD        145 non-null    float64
 7   AGED_DUAL_TOTAL_SPEND  272 non-null    float64
dtypes: float64(5), int64(2), object(1)
memory usage: 23.6+ KB


### B. Handle missing data

In [98]:
spending_columns = ['AGED_TOTAL_SPEND', 'DISABLED_TOTAL_SPEND', 'ESRD_TOTAL_ESRD', 'AGED_DUAL_TOTAL_SPEND']
df_total_cost_enroll[spending_columns] = df_total_cost_enroll[spending_columns].fillna(0)
df_total_cost_enroll.drop_duplicates(inplace=True)
df_total_cost_enroll.head()

,PERFORMANCE_YEAR,QRO,TIN_CCN,TOTAL_SPEND,AGED_TOTAL_SPEND,DISABLED_TOTAL_SPEND,ESRD_TOTAL_ESRD,AGED_DUAL_TOTAL_SPEND
0,2019,MTIP,122406,18332.78,1755.58,16577.20,0.00,0.00
1,2019,MTIP,679036,122.30,122.30,0.00,0.00,0.00
2,2019,MTIP,344888,8.79,0.00,0.00,8.79,0.00
3,2019,MTIP,125904,326079.83,165138.70,61145.21,99795.92,0.00
4,2019,MTIP,183834,51693176.93,36549230.25,7356027.20,3118860.85,4668087.62


### C. Convert QRO column to onehot encoding

In [108]:
one_hot_encoded = pd.get_dummies(df_total_cost_enroll['QRO'], prefix='QRO')
df_total_cost_enroll = df_total_cost_enroll.join(one_hot_encoded)
df_total_cost_enroll.head()

,PERFORMANCE_YEAR,QRO,TIN_CCN,TOTAL_SPEND,AGED_TOTAL_SPEND,DISABLED_TOTAL_SPEND,ESRD_TOTAL_ESRD,AGED_DUAL_TOTAL_SPEND,QRO_MKLO,QRO_MTIP,QRO_NTVP,QRO_OWTS,QRO_SDRU,QRO_UPLN
0,2019,MTIP,122406,18332.78,1755.58,16577.20,0.00,0.00,False,True,False,False,False,False
1,2019,MTIP,679036,122.30,122.30,0.00,0.00,0.00,False,True,False,False,False,False
2,2019,MTIP,344888,8.79,0.00,0.00,8.79,0.00,False,True,False,False,False,False
3,2019,MTIP,125904,326079.83,165138.70,61145.21,99795.92,0.00,False,True,False,False,False,False
4,2019,MTIP,183834,51693176.93,36549230.25,7356027.20,3118860.85,4668087.62,False,True,False,False,False,False


### D. Outlier detection

In [105]:
Q1_total_spend = df_total_cost_enroll['TOTAL_SPEND'].quantile(0.25)
Q3_total_spend = df_total_cost_enroll['TOTAL_SPEND'].quantile(0.75)
IQR_total_spend = Q3_total_spend - Q1_total_spend

lower_bound_total_spend = Q1_total_spend - 1.5 * IQR_total_spend
upper_bound_total_spend = Q3_total_spend + 1.5 * IQR_total_spend

outliers_total_spend = df_total_cost_enroll[(df_total_cost_enroll['TOTAL_SPEND'] < lower_bound_total_spend) | (df_total_cost_enroll['TOTAL_SPEND'] > upper_bound_total_spend)]

outliers_total_spend[['PERFORMANCE_YEAR', 'QRO', 'TIN_CCN', 'TOTAL_SPEND']]

,PERFORMANCE_YEAR,QRO,TIN_CCN,TOTAL_SPEND
4,2019,MTIP,183834,5.169318e+07
5,2019,MTIP,496926,2.383926e+07
6,2019,MTIP,834843,1.540325e+07
9,2019,NTVP,604033,4.316539e+07
15,2019,OWTS,355426,2.024740e+07
...,...,...,...,...
337,2023,OWTS,177085,1.278829e+07
340,2023,OWTS,659879,2.303950e+07
357,2023,SDRU,970310,5.993029e+07
358,2023,SDRU,971240,1.304199e+07


### E. Consistency checks

In [107]:
performance_year_check_cost = df_total_cost_enroll['PERFORMANCE_YEAR'].between(2019, 2023).all()

tin_ccn_unique_check_cost = df_total_cost_enroll['TIN_CCN'].is_unique

performance_year_check_cost, tin_ccn_unique_check_cost

if performance_year_check:
    print("All performace years between 2019 and 2023")
else:
    print("Found inconsistency in performace years. Not all in range 2019 to 2023")

if tin_ccn_unique_check:
    print("All CCN values are unique")
else:
    print("Not all CCN values are unique")

All performace years between 2019 and 2023
Not all CCN values are unique


## 3. Preprocessing for Tab: TIN_Q9_TotalCost_By_Categories

### A. Load sheet

In [117]:
df_total_cost_categories = pd.read_excel(file_path, sheet_name='TIN_Q9_TotalCost_By_Categories')
df_total_cost_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PERFORMANCE_YEAR     350 non-null    int64  
 1   QRO                  350 non-null    object 
 2   TIN_CCN              350 non-null    int64  
 3   SUM_IP_SPEND         291 non-null    float64
 4   SUM_FAC_OP_SPEND     349 non-null    float64
 5   SUM_PHY_SPEND        346 non-null    float64
 6   SUM_POSTACUTE_SPEND  253 non-null    float64
 7   SUM_HOSPICE_SPEND    227 non-null    float64
 8   SUM_DME_SPEND        295 non-null    float64
dtypes: float64(6), int64(2), object(1)
memory usage: 24.7+ KB


### B. Handle missing data

In [110]:
spending_columns_categories = ['SUM_IP_SPEND', 'SUM_FAC_OP_SPEND', 'SUM_PHY_SPEND', 'SUM_POSTACUTE_SPEND', 'SUM_HOSPICE_SPEND', 'SUM_DME_SPEND']
df_total_cost_categories[spending_columns_categories] = df_total_cost_categories[spending_columns_categories].fillna(0)

df_total_cost_categories.drop_duplicates(inplace=True)

df_total_cost_categories.head()

,PERFORMANCE_YEAR,QRO,TIN_CCN,SUM_IP_SPEND,SUM_FAC_OP_SPEND,SUM_PHY_SPEND,SUM_POSTACUTE_SPEND,SUM_HOSPICE_SPEND,SUM_DME_SPEND
0,2019,MTIP,183834,19192504.70,16020228.69,9062788.08,4477877.38,1320060.30,1349982.92
1,2019,MTIP,834843,5364179.15,4602899.62,3152931.99,1253099.57,433572.46,548635.03
2,2019,MTIP,122406,0.00,11329.40,7003.38,0.00,0.00,0.00
3,2019,MTIP,344888,0.00,8.79,0.00,0.00,0.00,0.00
4,2019,MTIP,125904,48225.71,134299.70,101493.16,12408.65,0.00,25392.39


### C. Convert QRO column to onehot encoding

In [111]:
one_hot_encoded = pd.get_dummies(df_total_cost_categories['QRO'], prefix='QRO')
df_total_cost_categories = df_total_cost_categories.join(one_hot_encoded)
df_total_cost_categories.head()

,PERFORMANCE_YEAR,QRO,TIN_CCN,SUM_IP_SPEND,SUM_FAC_OP_SPEND,SUM_PHY_SPEND,SUM_POSTACUTE_SPEND,SUM_HOSPICE_SPEND,SUM_DME_SPEND,QRO_MKLO,QRO_MTIP,QRO_NTVP,QRO_OWTS,QRO_SDRU,QRO_UPLN
0,2019,MTIP,183834,19192504.70,16020228.69,9062788.08,4477877.38,1320060.30,1349982.92,False,True,False,False,False,False
1,2019,MTIP,834843,5364179.15,4602899.62,3152931.99,1253099.57,433572.46,548635.03,False,True,False,False,False,False
2,2019,MTIP,122406,0.00,11329.40,7003.38,0.00,0.00,0.00,False,True,False,False,False,False
3,2019,MTIP,344888,0.00,8.79,0.00,0.00,0.00,0.00,False,True,False,False,False,False
4,2019,MTIP,125904,48225.71,134299.70,101493.16,12408.65,0.00,25392.39,False,True,False,False,False,False


### D. Outlier Detection

In [113]:
Q1_ip_spend = df_total_cost_categories['SUM_IP_SPEND'].quantile(0.25)
Q3_ip_spend = df_total_cost_categories['SUM_IP_SPEND'].quantile(0.75)
IQR_ip_spend = Q3_ip_spend - Q1_ip_spend

lower_bound_ip_spend = Q1_ip_spend - 1.5 * IQR_ip_spend
upper_bound_ip_spend = Q3_ip_spend + 1.5 * IQR_ip_spend

outliers_ip_spend = df_total_cost_categories[(df_total_cost_categories['SUM_IP_SPEND'] < lower_bound_ip_spend) | (df_total_cost_categories['SUM_IP_SPEND'] > upper_bound_ip_spend)]

outliers_ip_spend[['PERFORMANCE_YEAR', 'QRO', 'TIN_CCN', 'SUM_IP_SPEND']].head()

,PERFORMANCE_YEAR,QRO,TIN_CCN,SUM_IP_SPEND
0,2019,MTIP,183834,19192504.70
1,2019,MTIP,834843,5364179.15
6,2019,MTIP,496926,8349692.58
7,2019,NTVP,604033,14219144.68
24,2019,OWTS,355426,7826995.63


### E. Consistency checks

In [114]:
performance_year_check_categories = df_total_cost_categories['PERFORMANCE_YEAR'].between(2019, 2023).all()

tin_ccn_unique_check_categories = df_total_cost_categories['TIN_CCN'].is_unique

if performance_year_check:
    print("All performace years between 2019 and 2023")
else:
    print("Found inconsistency in performace years. Not all in range 2019 to 2023")

if tin_ccn_unique_check:
    print("All CCN values are unique")
else:
    print("Not all CCN values are unique")

All performace years between 2019 and 2023
Not all CCN values are unique


## 3. Preprocessing for Tab: TIN_Q10-13_Event_Rate

### A. Load sheet

In [116]:
df_event_rate = pd.read_excel(file_path, sheet_name='TIN_Q10-13_Event_Rate')
df_event_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PERFORMANCE_YEAR    314 non-null    int64  
 1   QRO                 314 non-null    object 
 2   TIN_CCN             314 non-null    int64  
 3   ED_VISIT_RATE       314 non-null    float64
 4   ED_RETURN_72HRS     287 non-null    float64
 5   IP_ADMIT_RATE       278 non-null    float64
 6   READMIT_30DAY_RATE  213 non-null    float64
dtypes: float64(4), int64(2), object(1)
memory usage: 17.3+ KB


### B. Handle missing data

In [130]:
event_columns_categories = ['ED_VISIT_RATE', 'ED_RETURN_72HRS', 'IP_ADMIT_RATE', 'READMIT_30DAY_RATE']
df_event_rate[event_columns_categories] = df_event_rate[event_columns_categories].fillna(0)

df_event_rate.drop_duplicates(inplace=True)

df_event_rate.head()

,PERFORMANCE_YEAR,QRO,TIN_CCN,ED_VISIT_RATE,ED_RETURN_72HRS,IP_ADMIT_RATE,READMIT_30DAY_RATE,QRO_MKLO,QRO_MTIP,QRO_NTVP,QRO_OWTS,QRO_SDRU,QRO_UPLN
0,2019,MTIP,183834,797.1,0.758,307.5,0.139,False,True,False,False,False,False
1,2019,MTIP,834843,688.6,0.696,230.1,0.094,False,True,False,False,False,False
2,2019,MTIP,122406,166.7,1.000,0.0,0.000,False,True,False,False,False,False
3,2019,MTIP,125904,1062.5,0.735,218.8,0.000,False,True,False,False,False,False
4,2019,MTIP,496926,1004.5,0.779,354.0,0.145,False,True,False,False,False,False


### C. Convert QRO column to onehot encoding

In [121]:
one_hot_encoded = pd.get_dummies(df_event_rate['QRO'], prefix='QRO')
df_event_rate = df_event_rate.join(one_hot_encoded)
df_event_rate.head()

,PERFORMANCE_YEAR,QRO,TIN_CCN,ED_VISIT_RATE,ED_RETURN_72HRS,IP_ADMIT_RATE,READMIT_30DAY_RATE,QRO_MKLO,QRO_MTIP,QRO_NTVP,QRO_OWTS,QRO_SDRU,QRO_UPLN
0,2019,MTIP,183834,797.1,0.758,307.5,0.139,False,True,False,False,False,False
1,2019,MTIP,834843,688.6,0.696,230.1,0.094,False,True,False,False,False,False
2,2019,MTIP,122406,166.7,1.000,0.0,0.000,False,True,False,False,False,False
3,2019,MTIP,125904,1062.5,0.735,218.8,0.000,False,True,False,False,False,False
4,2019,MTIP,496926,1004.5,0.779,354.0,0.145,False,True,False,False,False,False


### D. Outlier Detection

In [123]:
Q1_ip_spend = df_event_rate['ED_VISIT_RATE'].quantile(0.25)
Q3_ip_spend = df_event_rate['ED_VISIT_RATE'].quantile(0.75)
IQR_ip_spend = Q3_ip_spend - Q1_ip_spend

lower_bound_ip_spend = Q1_ip_spend - 1.5 * IQR_ip_spend
upper_bound_ip_spend = Q3_ip_spend + 1.5 * IQR_ip_spend

outliers_ip_spend = df_event_rate[(df_event_rate['ED_VISIT_RATE'] < lower_bound_ip_spend) | (df_event_rate['ED_VISIT_RATE'] > upper_bound_ip_spend)]

outliers_ip_spend[['ED_VISIT_RATE']].head()

,ED_VISIT_RATE
7,1576.9
15,1201.2
25,2238.1
64,1555.6
74,1250.0


In [124]:
Q1_ip_spend = df_event_rate['ED_RETURN_72HRS'].quantile(0.25)
Q3_ip_spend = df_event_rate['ED_RETURN_72HRS'].quantile(0.75)
IQR_ip_spend = Q3_ip_spend - Q1_ip_spend

lower_bound_ip_spend = Q1_ip_spend - 1.5 * IQR_ip_spend
upper_bound_ip_spend = Q3_ip_spend + 1.5 * IQR_ip_spend

outliers_ip_spend = df_event_rate[(df_event_rate['ED_RETURN_72HRS'] < lower_bound_ip_spend) | (df_event_rate['ED_RETURN_72HRS'] > upper_bound_ip_spend)]

outliers_ip_spend[['ED_RETURN_72HRS']].head()

,ED_RETURN_72HRS


In [125]:
Q1_ip_spend = df_event_rate['IP_ADMIT_RATE'].quantile(0.25)
Q3_ip_spend = df_event_rate['IP_ADMIT_RATE'].quantile(0.75)
IQR_ip_spend = Q3_ip_spend - Q1_ip_spend

lower_bound_ip_spend = Q1_ip_spend - 1.5 * IQR_ip_spend
upper_bound_ip_spend = Q3_ip_spend + 1.5 * IQR_ip_spend

outliers_ip_spend = df_event_rate[(df_event_rate['IP_ADMIT_RATE'] < lower_bound_ip_spend) | (df_event_rate['IP_ADMIT_RATE'] > upper_bound_ip_spend)]

outliers_ip_spend[['IP_ADMIT_RATE']].head()

,IP_ADMIT_RATE
12,562.8
25,1809.5
50,809.5
64,1444.4
97,561.8


In [126]:
Q1_ip_spend = df_event_rate['READMIT_30DAY_RATE'].quantile(0.25)
Q3_ip_spend = df_event_rate['READMIT_30DAY_RATE'].quantile(0.75)
IQR_ip_spend = Q3_ip_spend - Q1_ip_spend

lower_bound_ip_spend = Q1_ip_spend - 1.5 * IQR_ip_spend
upper_bound_ip_spend = Q3_ip_spend + 1.5 * IQR_ip_spend

outliers_ip_spend = df_event_rate[(df_event_rate['READMIT_30DAY_RATE'] < lower_bound_ip_spend) | (df_event_rate['READMIT_30DAY_RATE'] > upper_bound_ip_spend)]

outliers_ip_spend[['READMIT_30DAY_RATE']].head()

,READMIT_30DAY_RATE
25,0.368
95,0.600
177,0.406
192,0.400


### E. Consistency checks

In [127]:
performance_year_check_categories = df_event_rate['PERFORMANCE_YEAR'].between(2019, 2023).all()

tin_ccn_unique_check_categories = df_event_rate['TIN_CCN'].is_unique

if performance_year_check:
    print("All performace years between 2019 and 2023")
else:
    print("Found inconsistency in performace years. Not all in range 2019 to 2023")

if tin_ccn_unique_check:
    print("All CCN values are unique")
else:
    print("Not all CCN values are unique")

All performace years between 2019 and 2023
Not all CCN values are unique


## 3. Preprocessing for Tab: TIN_VISIT_Q8-14

In [129]:
df_tin_visit = pd.read_excel(file_path, sheet_name='TIN_VISIT_Q8-14')
df_tin_visit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PERFORMANCE_YEAR  352 non-null    int64  
 1   QRO               352 non-null    object 
 2   TIN_CCN           352 non-null    int64  
 3   AWV_RATE          352 non-null    float64
 4   SUM_PCP_VISITS    352 non-null    float64
 5   CNT_PCP_VISITS    352 non-null    int64  
 6   SUM_SPEC_VISITS   352 non-null    float64
 7   CNT_SPEC_VISITS   352 non-null    int64  
dtypes: float64(3), int64(4), object(1)
memory usage: 22.1+ KB


### B. Handle missing data

In [134]:
event_columns_categories = ['AWV_RATE', 'SUM_PCP_VISITS', 'CNT_PCP_VISITS', 'SUM_SPEC_VISITS', 'CNT_SPEC_VISITS']

df_tin_visit[event_columns_categories] = df_tin_visit[event_columns_categories].fillna(0)

df_tin_visit.drop_duplicates(inplace=True)

df_tin_visit.head()

,PERFORMANCE_YEAR,QRO,TIN_CCN,AWV_RATE,SUM_PCP_VISITS,CNT_PCP_VISITS,SUM_SPEC_VISITS,CNT_SPEC_VISITS
0,2019,MTIP,122406,0.000,988.59,10,460.16,12
1,2019,MTIP,679036,0.000,122.30,1,0.00,0
2,2019,MTIP,344888,0.000,8.79,1,0.00,0
3,2019,MTIP,166870,0.000,0.00,0,0.00,0
4,2019,MTIP,125904,0.438,26994.97,260,9199.88,153


### C. Convert QRO column to onehot encoding

In [135]:
one_hot_encoded = pd.get_dummies(df_tin_visit['QRO'], prefix='QRO')
df_tin_visit = df_tin_visit.join(one_hot_encoded)
df_tin_visit.head()

,PERFORMANCE_YEAR,QRO,TIN_CCN,AWV_RATE,SUM_PCP_VISITS,CNT_PCP_VISITS,SUM_SPEC_VISITS,CNT_SPEC_VISITS,QRO_MKLO,QRO_MTIP,QRO_NTVP,QRO_OWTS,QRO_SDRU,QRO_UPLN
0,2019,MTIP,122406,0.000,988.59,10,460.16,12,False,True,False,False,False,False
1,2019,MTIP,679036,0.000,122.30,1,0.00,0,False,True,False,False,False,False
2,2019,MTIP,344888,0.000,8.79,1,0.00,0,False,True,False,False,False,False
3,2019,MTIP,166870,0.000,0.00,0,0.00,0,False,True,False,False,False,False
4,2019,MTIP,125904,0.438,26994.97,260,9199.88,153,False,True,False,False,False,False


### D. Outlier Detection

In [136]:
Q1_ip_spend = df_tin_visit['AWV_RATE'].quantile(0.25)
Q3_ip_spend = df_tin_visit['AWV_RATE'].quantile(0.75)
IQR_ip_spend = Q3_ip_spend - Q1_ip_spend

lower_bound_ip_spend = Q1_ip_spend - 1.5 * IQR_ip_spend
upper_bound_ip_spend = Q3_ip_spend + 1.5 * IQR_ip_spend

outliers_ip_spend = df_tin_visit[(df_tin_visit['AWV_RATE'] < lower_bound_ip_spend) | (df_tin_visit['AWV_RATE'] > upper_bound_ip_spend)]

outliers_ip_spend[['AWV_RATE']].head()

,AWV_RATE


In [137]:
Q1_ip_spend = df_tin_visit['SUM_PCP_VISITS'].quantile(0.25)
Q3_ip_spend = df_tin_visit['SUM_PCP_VISITS'].quantile(0.75)
IQR_ip_spend = Q3_ip_spend - Q1_ip_spend

lower_bound_ip_spend = Q1_ip_spend - 1.5 * IQR_ip_spend
upper_bound_ip_spend = Q3_ip_spend + 1.5 * IQR_ip_spend

outliers_ip_spend = df_tin_visit[(df_tin_visit['SUM_PCP_VISITS'] < lower_bound_ip_spend) | (df_tin_visit['SUM_PCP_VISITS'] > upper_bound_ip_spend)]

outliers_ip_spend[['SUM_PCP_VISITS']].head()

,SUM_PCP_VISITS
5,1464784.77
6,1244567.92
7,954668.84
10,3345490.93
16,686899.97


In [138]:
Q1_ip_spend = df_tin_visit['CNT_PCP_VISITS'].quantile(0.25)
Q3_ip_spend = df_tin_visit['CNT_PCP_VISITS'].quantile(0.75)
IQR_ip_spend = Q3_ip_spend - Q1_ip_spend

lower_bound_ip_spend = Q1_ip_spend - 1.5 * IQR_ip_spend
upper_bound_ip_spend = Q3_ip_spend + 1.5 * IQR_ip_spend

outliers_ip_spend = df_tin_visit[(df_tin_visit['CNT_PCP_VISITS'] < lower_bound_ip_spend) | (df_tin_visit['CNT_PCP_VISITS'] > upper_bound_ip_spend)]

outliers_ip_spend[['CNT_PCP_VISITS']].head()

,CNT_PCP_VISITS
5,23075
6,10951
7,8647
10,25061
16,9064


In [139]:
Q1_ip_spend = df_tin_visit['SUM_SPEC_VISITS'].quantile(0.25)
Q3_ip_spend = df_tin_visit['SUM_SPEC_VISITS'].quantile(0.75)
IQR_ip_spend = Q3_ip_spend - Q1_ip_spend

lower_bound_ip_spend = Q1_ip_spend - 1.5 * IQR_ip_spend
upper_bound_ip_spend = Q3_ip_spend + 1.5 * IQR_ip_spend

outliers_ip_spend = df_tin_visit[(df_tin_visit['SUM_SPEC_VISITS'] < lower_bound_ip_spend) | (df_tin_visit['SUM_SPEC_VISITS'] > upper_bound_ip_spend)]

outliers_ip_spend[['SUM_SPEC_VISITS']].head()

,SUM_SPEC_VISITS
5,1050828.72
6,471142.99
7,263161.19
10,691382.89
16,620734.70


In [140]:
Q1_ip_spend = df_tin_visit['CNT_SPEC_VISITS'].quantile(0.25)
Q3_ip_spend = df_tin_visit['CNT_SPEC_VISITS'].quantile(0.75)
IQR_ip_spend = Q3_ip_spend - Q1_ip_spend

lower_bound_ip_spend = Q1_ip_spend - 1.5 * IQR_ip_spend
upper_bound_ip_spend = Q3_ip_spend + 1.5 * IQR_ip_spend

outliers_ip_spend = df_tin_visit[(df_tin_visit['CNT_SPEC_VISITS'] < lower_bound_ip_spend) | (df_tin_visit['CNT_SPEC_VISITS'] > upper_bound_ip_spend)]

outliers_ip_spend[['CNT_SPEC_VISITS']].head()

,CNT_SPEC_VISITS
5,17304
6,7624
7,4254
10,10660
16,8926


### E. Consistency checks

In [141]:
performance_year_check_categories = df_tin_visit['PERFORMANCE_YEAR'].between(2019, 2023).all()

tin_ccn_unique_check_categories = df_tin_visit['TIN_CCN'].is_unique

if performance_year_check:
    print("All performace years between 2019 and 2023")
else:
    print("Found inconsistency in performace years. Not all in range 2019 to 2023")

if tin_ccn_unique_check:
    print("All CCN values are unique")
else:
    print("Not all CCN values are unique")

All performace years between 2019 and 2023
Not all CCN values are unique
